<a href="https://colab.research.google.com/github/chh172/CSE100-PA1/blob/master/max_cut_in_Pointer%26Actor_Critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
from networkx import *
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import keras.backend as K
# from keras.engine import InputSpec
from tensorflow.keras.layers import InputSpec
from keras.activations import tanh, softmax
from keras.layers import LSTM

# copyright to https://github.com/keon/pointer-networks
class Attention(keras.layers.Layer):
    """
        Attention layer
    """

    def __init__(self, hidden_dimensions, name='attention'):
        super(Attention, self).__init__(name=name, trainable=True)
        self.W1 = keras.layers.Dense(hidden_dimensions, use_bias=False)
        self.W2 = keras.layers.Dense(hidden_dimensions, use_bias=False)
        self.V = keras.layers.Dense(1, use_bias=False)

    def call(self, encoder_outputs, dec_output, mask=None):

        w1_e = self.W1(encoder_outputs)
        w2_d = self.W2(dec_output)
        tanh_output = tanh(w1_e + w2_d)
        v_dot_tanh = self.V(tanh_output)
        print(v_dot_tanh.shape)
        if mask is not None:
            v_dot_tanh += (mask * -1e9)
        attention_weights = softmax(v_dot_tanh, axis=1)
        att_shape = K.shape(attention_weights)
        return K.reshape(attention_weights, (att_shape[0], att_shape[1]))


class Decoder(keras.layers.Layer):
    """
        Decoder class for PointerLayer
    """

    def __init__(self, hidden_dimensions):
        super(Decoder, self).__init__()
        self.lstm = keras.layers.LSTM(
            hidden_dimensions, return_sequences=False, return_state=True)

    def call(self, x, hidden_states):
        dec_output, state_h, state_c = self.lstm(
            x, initial_state=hidden_states)
        return dec_output, [state_h, state_c]

    def get_initial_state(self, inputs):
        return self.lstm.get_initial_state(inputs)

    def process_inputs(self, x_input, initial_states, constants):
        return self.lstm._process_inputs(x_input, initial_states, constants)


class PointerLSTM(keras.layers.Layer):
    """
        PointerLSTM
    """

    def __init__(self, hidden_dimensions, name='pointer', **kwargs):
        super(PointerLSTM, self).__init__(**kwargs)
        self.hidden_dimensions = hidden_dimensions
        self.attention = Attention(hidden_dimensions)
        self.decoder = Decoder(hidden_dimensions)

    def build(self, input_shape):
        super(PointerLSTM, self).build(input_shape)
        self.input_spec = [InputSpec(shape=input_shape)]

    def call(self, x, training=None, mask=None, states=None):
        """
        :param Tensor x: Should be the output of the decoder
        :param Tensor states: last state of the decoder
        :param Tensor mask: The mask to apply
        :return: Pointers probabilities
        """

        input_shape = self.input_spec[0].shape
        en_seq = x
        x_input = x[:, input_shape[1] - 1, :]
        x_input = K.repeat(x_input, input_shape[1])
        if states:
            initial_states = states
        else:
            initial_states = self.decoder.get_initial_state(x_input)

        constants = []
        preprocessed_input, _, constants = self.decoder.process_inputs(
            x_input, initial_states, constants)
        constants.append(en_seq)
        last_output, outputs, states = K.rnn(self.step, preprocessed_input,
                                             initial_states,
                                             go_backwards=self.decoder.lstm.go_backwards,
                                             constants=constants,
                                             input_length=input_shape[1])

        return outputs

    def step(self, x_input, states):
        x_input = K.expand_dims(x_input,1)
        input_shape = self.input_spec[0].shape
        en_seq = states[-1]
        _, [h, c] = self.decoder(x_input, states[:-1])
        dec_seq = K.repeat(h, input_shape[1])
        probs = self.attention(dec_seq, en_seq)
        return probs, [h, c]

    def get_output_shape_for(self, input_shape):
        # output shape is not affected by the attention component
        return (input_shape[0], input_shape[1], input_shape[1])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[1])

In [3]:
import random 
from networkx import *
n = 6
p = 0.5
g = erdos_renyi_graph(n, p)
print(g. nodes)
# [0, 1, 2, 3, 4, 5] 
print(g. edges)
# [(0, 1), (0, 2), (0, 4), (1, 2), (1, 5), (3, 4), (4, 5)]

[0, 1, 2, 3, 4, 5]
[(0, 1), (0, 2), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 5), (3, 4), (3, 5), (4, 5)]


In [6]:
# Description: this function randomly generate a sequence of graphs G(V,E)
# where graphs are represented by their adjacency matrix A
# and the distribution of V and E are all uniform.
# It takes in 'number' of graphs, 'infimum' and 'supremum' of |V|. 
import numpy as np
from networkx import *
import random
def graph_seq_generator(num, inf, sup):
  atlas = []
  for i in range(num):
    A = adjacency_matrix(erdos_renyi_graph(np.random.randint(inf,sup), 0.5))
    #print(A.todense())
    atlas.append(A.todense())
  return atlas    



In [5]:
import numpy as np
# Description: this function tensor-ize the atlas, preparing for LSTM
def input_generator(atlas):
  return tf.cast(np.array(random.sample(atlas,1)),tf.float32)

In [3]:
# Description: this function randomly generate a random graph G(V,E)
# where graph is represented by their adjacency matrix A

def graph_generator(vertices):
  return tf.cast(np.array([adjacency_matrix(erdos_renyi_graph(vertices, 0.5)).todense()]),tf.float32)


In [3]:
# Unit testing
# atlas = graph_seq_generator(4,1,5)
#print(atlas[1])
#inp = Input(atlas[1])
#print(inp)
#for i in range(3):
  #print(g[i].nodes)
  #
  #print(g[i].edges)
  #
  #print(len(g[i].nodes))
#g = random.sample(atlas,2)
#print(g[1])
#input = input_generator(atlas)
#input_2 = tf.cast(np.array([atlas[1]]),tf.float32)
#print(input)
#print(input_2)
#print(input.shape[0])
#print(input.shape[1])
#print(input.shape[2])
#print(input.shape)
#print(input.shape)
# x = Input(shape=(32,))
#x_1 = Input(input)
# print(x)
# print(graph_generator(5))
# print(tf.keras.Input(shape=(10,None,5),tensor=graph_generator(5)))
def increment(x):
  x = x +1
  return
x = 1
increment(x)
print(x)  

1


In [15]:
# actor-critic training
# starting state: (0,0,...,0)
# action space = {action: flip a single bit in state} U {ternimate}
def actor_critic_algo(actor,critic,graph):
  # randomly init 'phi', 'theta'
  # do until theta converges
    # init starting state
    # \lambda = 1
    # do until reach terminal state
      # In state 's', select action 'a' given by actor
      # perform 'a' and collect reward r and new state (terminal state)
      # update delta
      # update value function parameter 'phi'
      # update policy parameter 'theta'
      # update discount 
      # update state
  return

def delta_update(critic,reward):
  return 

def value_f_update(critic, delta ,phi, beta):
  return phi+ beta*delta*value_grad(critic,phi)

def policy_update(actor,alpha,lamb,delta,theta):
  return theta + alpha*lamb*delta*policy_grad(actor,theta)
#  
def policy_grad(actor,theta):
  with tf.GradientTape() as tape:
    tape.watch(theta)
    y = tf.math.log(actor(theta))
  return tape.gradient(y,theta) 
def value_grad(critic,phi):
  with tf.GradientTape() as tape:
    tape.watch(phi)
    y = critic(phi)
  return tape.gradient(y, phi) 

In [14]:

# training main in way 2

hidden_size = 128
vertices_size = 5

# input is a [A], where A is the adjacency matrix, which is a square matrix
# sequence length shall match the number of rows and feature_dim is number of cols
# note the feature for each node is its adjacency relation (row vector in adjacency matrix)
seq_len = vertices_size
feature_dim = vertices_size

encoder = LSTM(hidden_size,return_sequences = True, name="encoder",return_state=True)
actor_decoder = PointerLSTM( hidden_size, name="actor_decoder")
critic_decoder = LSTM(hidden_size,name="critic_decoder")

inputs = keras.layers.Input(shape=(seq_len, feature_dim)) 
encoder_o, state_h, state_c = encoder(inputs)
policy = actor_decoder(encoder_o,states=[state_h, state_c])
scores = critic_decoder(encoder_o)



#actor = tf.keras.Model(inputs=inputs, outputs=policy)

#critic = tf.keras.Model(inputs=inputs, outputs=scores)
model = tf.keras.Model(inputs=inputs, outputs=[policy,scores])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.Huber(),
              metrics=['accuracy'])
graph = graph_generator(5)
with tf.GradientTape() as tape:
  tape.watch(graph)
  y_p, y_c= model(graph)

grad = tape.gradient(y_p,graph)
print(grad)
# for i in range(5):
graph = graph_generator(5)
print(graph)
encoder_outputs, state_h, state_c = encoder(graph)
#print(encoder_outputs)
print("hello world")
policy = actor_decoder(encoder_outputs)
print(policy)
scores = critic_decoder(encoder_outputs)
print("hello world")
print(scores)


(None, 5, 1)
(None, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
tf.Tensor(
[[[ 6.0458760e-10 -7.5285472e-11  4.8421123e-10 -1.1959041e-10
   -4.2477780e-10]
  [ 7.6868217e-10 -1.2971360e-10  4.7492005e-10 -2.0694016e-10
   -5.2709648e-10]
  [ 9.6264707e-10 -2.6364563e-10  6.0354527e-10 -2.3108065e-10
   -6.8260475e-10]
  [ 1.2969970e-09 -4.3170742e-10  5.7867711e-10 -3.6762932e-10
   -8.2845975e-10]
  [ 1.5167826e-09 -3.5578995e-10  6.1202021e-10 -6.7534517e-10
   -9.7888719e-10]]], shape=(1, 5, 5), dtype=float32)
tf.Tensor(
[[[0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 0.]
  [1. 0. 0. 0. 1.]
  [1. 1. 0. 0. 0.]
  [1. 0. 1. 0. 0.]]], shape=(1, 5, 5), dtype=float32)
hello world
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
(1, 5, 1)
tf.Tensor(
[[[0.19770537 0.19880718 0.20375657 0.20056958 0.19916134]
  [0.19770473 0.19880463 0.20375726 0.20057082 0.19916257]
  [0.19770396 0.19880229 0.20375772 0.20057203 0.19916405]
  [0.19770327 0.19880036 0.20375797 0.200573   0.19916546

In [10]:
import keras
keras.__version__

'2.8.0'

In [8]:


# training main in way 1

actor_encoder = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# pointer works as decoder
pointer = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
# critic/value works as encoder
critic = tf.keras.layers.LSTM(128, return_sequences=True, return_state=True)
inputs = tf.keras.Input(shape=(None,5)) # vertices size 5
encoder_outputs, state_h, state_c = actor_encoder(inputs)
decoder_outputs, de_state_h, de_state_c = pointer(encoder_outputs)
# TODO: ADD SOFTMAX as classifier, then feed to critic
value, critic_state_h, critic_state_c = critic(decoder_outputs)
actor = tf.keras.Model(inputs=inputs, outputs=decoder_outputs)
model = tf.keras.Model(inputs=inputs, outputs=value)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.Huber(),
              metrics=['accuracy'])
graph = graph_generator(5)
with tf.GradientTape() as tape:
  tape.watch(graph)
  y = model(graph)

grad = tape.gradient(y,graph)
print(grad)
# TODO: Specify the functioning of pointer network (in detail)
# train it such that pointer returns "sequence" of indices from largest probability to lowest

for i in range(5):
  graph = graph_generator(5)
  #print(graph)
  actor_outputs, state_h, state_c = actor_encoder(graph)
  decoder_outputs, de_state_h, de_state_c = pointer(actor_outputs)
  value, critic_state_h, critic_state_c = critic(decoder_outputs)


tf.Tensor(
[[[-1.9091982e-03  6.2075101e-02 -1.0138596e-01  8.7714098e-02
    3.4673616e-02]
  [ 3.0340068e-03  4.3968163e-02 -6.6788293e-02  6.0502719e-02
    2.0162243e-02]
  [ 1.8023690e-03  2.8962709e-02 -3.7919164e-02  3.0463574e-02
    1.4514055e-02]
  [ 6.7431107e-04  1.8162895e-02 -1.5985496e-02  1.3181592e-02
    1.0792833e-02]
  [ 1.0046002e-04  5.1129898e-03 -4.4046668e-03  3.7885746e-03
    2.9451679e-03]]], shape=(1, 5, 5), dtype=float32)


In [9]:
# (inference template) main

#atlas = graph_seq_generator(1,1,5)
graph = graph_generator(5)
#print(graph)
# actor/policy works as encoder
actor = tf.keras.layers.LSTM(8, return_sequences=True, return_state=True)
# pointer works as decoder
pointer = tf.keras.layers.LSTM(8, return_sequences=True, return_state=True)
# critic/value works as encoder
critic = tf.keras.layers.LSTM(8, return_sequences=True, return_state=True)
actor_outputs, state_h, state_c = actor(graph)
decoder_outputs, de_state_h, de_state_c = pointer(actor_outputs)
value, critic_state_h, critic_state_c = critic(decoder_outputs)
#print(value)
#print(actor_outputs)
print(decoder_outputs)

tf.Tensor(
[[[ 0.00545292  0.00019841  0.00227465  0.00554217 -0.01269028
   -0.00623421  0.01094057  0.00446605]
  [ 0.00964246  0.00209888  0.00357724  0.00762778 -0.01860836
   -0.0099867   0.01560443  0.00614076]
  [ 0.02958955  0.01480201 -0.00704743  0.0066876  -0.03216184
   -0.01166113  0.01171903 -0.01194777]
  [ 0.03616894  0.00810451 -0.01115538 -0.00103414 -0.04054455
   -0.01983783 -0.01930784 -0.02853147]
  [ 0.05480311  0.01469139 -0.02617485 -0.00451238 -0.04847791
   -0.02508336 -0.03667019 -0.0415091 ]]], shape=(1, 5, 8), dtype=float32)
